In [1]:
import pandas as pd
from statistics import mean, median
import sys
import pprint

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
old_result_file = 'median_sbs_delay_all_methods.csv'
ml_delay_file = '../RQ3/final_results/'
ssr_result_file = '../RQ2/gfm_all_ssr_results.csv'

In [4]:
batch_sizes = [2, 4, 8, 16]
batching_alg = ['BATCH4', 'BATCHBISECT', 'BATCHSTOP4']

In [5]:
old = pd.read_csv(old_result_file, index_col=0)
gr_results = old[ (old['method']=='gr') & (old['batch_size'] != 32) & ( old['batch_size'] != 1 )]
projects = set(gr_results['project'])

In [ ]:
for p in projects:
    
    
    test_file = pd.read_csv('../data/test_data/'+p.split('.')[0]+'_test.csv')
    ml_results = pd.read_csv(ml_delay_file + p.split('.')[0] + '_ml_batching.csv')
    
    length = len(test_file)
    
    #ml batch median delay calculation
    for x in range(len(ml_results)):
        row = ml_results.iloc[x]
        ci = str_to_list(row['ci'])
        b = row['batch_size']
        
        #gr batch median delay calculation
        gr_ci = [0] * length
        i = 0
        gr_delay_list = []
        while i < len(gr_ci):
            b -= 1
            gr_delay_list.append(b)
            if b == 0:
                b = row['batch_size']
            i += 1
        
        gr_delay_lib[p][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(gr_delay_list)
        
        i = 0
        delay_list = []
        while i < len(ci):
            if ci[i] == 0:
                if b >= 1:
                    b -= 1
                    delay_list.append(b)
                elif b == 0:
                    b = row['batch_size']
                    b -= 1
                    delay_list.append(b)
            if ci[i] == 1:
                delay_list.append(0)
                b = row['batch_size']
            i += 1
        ml_delay_lib[p][row['algorithm']][row['batch_size']]['batch_median_delay'] = median(delay_list)